<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/exercices-solutions/03_Computer_Vision_and_Neural_Network_Exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 - Computer Vision and Neural Networks Exercices

Take a dataset on kaggle (ex: cat vs dog) and Build a CNN image classifier using the techniques we've built in the course notebook. 

Find an ideal learning rate for a simple convolutional neurol network model on the dataset.